In [1]:
import pandas as pd # library for data analysis
import requests # library to handle requests
from py2neo import Node, Graph, Relationship, NodeMatcher
from py2neo.bulk import merge_nodes
from bs4 import BeautifulSoup # library to parse HTML documents

In [2]:
from neo4j import GraphDatabase

# Initialize connection to database
driver = GraphDatabase.driver('bolt://localhost:7687', auth=('neo4j', 'pyqt5858!'))
session = driver.session()

In [3]:
#  해군관련된 url 만들기

wikiurl='https://ko.wikipedia.org/wiki/%EB%8C%80%ED%95%9C%EB%AF%BC%EA%B5%AD_%ED%95%B4%EA%B5%B0'
response=requests.get(wikiurl)
base_url='https://ko.wikipedia.org'

soup = BeautifulSoup(response.text, 'html.parser')
indiatable=soup.find('table',{'class':"navbox-inner"})

url_list=[]
if indiatable:
    a_tags = indiatable.find_all('a')
    for a_tag in a_tags:
        url = a_tag.get('href')
        if url:
            url_list.append(base_url+url)
else:
    print("해당 클래스를 가진 div 태그가 존재하지 않습니다.")


In [4]:
# get the response in the form of html
def crawling(url):
    wikiurl=url
    response=requests.get(wikiurl)
    # parse data from the html into a beautifulsoup object
    soup = BeautifulSoup(response.text, 'html.parser')
    indiatable=soup.find('table',{'class':"infobox"})

    df=pd.read_html(str(indiatable))
    # convert list to dataframe
    df=pd.DataFrame(df[0])
    return df

def dataframe_to_tuples(df):
    tuples_list = [tuple(row) for _, row in df.iterrows()]
    return tuples_list

def make_prop_rel(list):
    property_list=['함명', '함번', '진수', '배치', '배수량', '전장', '선폭', '흘수', '속력', '항속거리', '승조원']  #여기서 함정에 대한 property를 정의
    property_list=[]
    relation_list=[]
    for tup in list:
        if tup[0] in property_list:
            property_list.append(tup)
        else:
            relation_list.append(tup)
    return property_list, relation_list

#함정에서 node 생성하기 : 라벨은(tuble[0], tuple[1]) / 있으면 relation만 없으면 node도 생성
#함정에서 relation 생성하기 : relation 이름은 tuple[0]

In [5]:
#생성하기
def add_nodes(ls):
    result='create(n:함정{'
    for i in range(len(ls)-1):
        properties=str(ls[i][0])+':"'+str(ls[i][1])+'",'
        result+=properties
    print('add_nodes')
    return result + str(ls[-1][0])+':"'+str(ls[-1][1]) + '"})'

#match여부 확인
def match_nodes(ls):
    from neo4j import GraphDatabase
    for i in ls:
        if i[0]=='함명':
            result=i[1]
    query='match (n{함명:"'+result+'"}) return n'
    driver = GraphDatabase.driver('bolt://localhost:7687', auth=('neo4j', 'pyqt5858!'))
    session = driver.session()
    results=list(session.run(query))
    if results:
        return True
    else:
        return False
    
#match결과 없을경우 add node
def match_add(url):
    ls=crawling(url)
    ls=dataframe_to_tuples(ls)
    prob, rel=make_prop_rel(ls)
    if match_nodes(prob) != True:
        session.run(add_nodes(ls))
        
        


In [6]:
for i in url_list:
    try:
        match_add(i)
        print(i, '성공')
    except:
        print(i, '실패')

https://ko.wikipedia.org/wiki/%ED%8B%80:%EB%8C%80%ED%95%9C%EB%AF%BC%EA%B5%AD_%ED%95%B4%EA%B5%B0_%EC%9E%A5%EB%B9%84 실패
https://ko.wikipedia.org/wiki/%ED%8B%80%ED%86%A0%EB%A1%A0:%EB%8C%80%ED%95%9C%EB%AF%BC%EA%B5%AD_%ED%95%B4%EA%B5%B0_%EC%9E%A5%EB%B9%84 실패
https://ko.wikipedia.orghttps://ko.wikipedia.org/w/index.php?title=%ED%8B%80:%EB%8C%80%ED%95%9C%EB%AF%BC%EA%B5%AD_%ED%95%B4%EA%B5%B0_%EC%9E%A5%EB%B9%84&action=edit 실패
                                                 대한민국  \
0                                                大韓民國   
1                                            국기 국장 국새   
2                                                  국기   
3                                                  국새   
4                                                  표어   
5                                                  국가   
6                                                 NaN   
7                                                  수도   
8   .mw-parser-output .locmap .od{position:absolut...   
9        

In [ ]:
#노드를 찾은담에 relation 추가
#무